In [21]:
from bb_behavior import plot
from bb_behavior import db as bbdb
import bb_behavior
import pandas as pd
import os
import sys
sys.path.append('/home/mi/rrszynka/mnt/janek/Beesbook-janek/Python-modules/') #For bee_helpers, file_helpers and cache
from bee_cache import Cache, CacheType, CacheFormat; c = Cache()
import datetime
from skimage.morphology import rectangle, closing
import datetime
import numpy as np
from tqdm import tqdm
import random
#%%

In [22]:
# detection_confidence_requirement = 0.99
detection_confidence_requirement = 0.2
conf_string = str(detection_confidence_requirement).replace('.',"")
interval_size = 30
fps = 3

loc = '/home/mi/rrszynka/mnt/janek/caches/Other/'

# Get one of the pre-generated lists  
gaps = pd.read_csv(loc+'120_random_gaps_unmasked_unfiltered_seed_666_conf_' + conf_string + '.csv', parse_dates=['entry', 'entry_closest_detection_timestamp', 'exit', 'exit_closest_detection_timestamp'])

In [23]:
%%capture
# Load presence files to be able to refer to them when generating vids 
presences = c.load_multiple_day_caches(detection_confidence_requirement=detection_confidence_requirement) # 0.99 / 0.2

In [24]:
presences_bin = []
for date, presence in presences:
    presence_bin = binarize_presence(presence)
    presences_bin.append((date, presence_bin))

In [25]:
#TODO: from 120_random_gaps_unmasked_unfiltered_seed_666_conf_099, .iloc[14] doesn't work (something is nan)A

In [26]:
row = gaps.iloc[17]
# generate_presence_video(row)

In [27]:
row

Unnamed: 0                                                18569
bee_id                                                     2051
age                                                           1
age_group                                                     0
exit                                        2016-07-26 10:57:30
entry                                       2016-07-26 11:08:30
gap_duration                          0 days 00:11:00.000000000
exit_x                                                  194.517
exit_y                                                  32.5805
exit_frame                                  9587963831926512918
exit_closest_detection_timestamp     2016-07-26 10:57:29.833820
cam_id_exit                                                   3
hive_side_exit                                                1
entry_x                                                 195.849
entry_y                                                 93.2848
entry_frame                             

In [28]:
gaps

,Unnamed: 0,bee_id,age,age_group,exit,entry,gap_duration,exit_x,exit_y,exit_frame,...,entry_x,entry_y,entry_frame,entry_closest_detection_timestamp,cam_id_entry,hive_side_entry,origin_for_exit,origin_for_entry,hiveexit_distance_for_exit,hiveexit_distance_for_entry
0,22322,2175,4.0,0,2016-07-31 02:52:00,2016-07-31 03:05:00,0 days 00:13:00.000000000,199.558100,70.186670,17029705002092385638,...,195.026000,83.099680,10110498357781109175,2016-07-31 03:05:00.008879,3,1,[ 0 250],[ 0 250],268.619189,256.692145
1,14514,1395,8.0,1,2016-07-23 04:48:30,2016-07-23 05:01:30,0 days 00:13:00.000000000,183.097900,53.210090,10822678284860735268,...,96.365620,39.584000,17141597838569932581,2016-07-23 05:02:10.437360,2,1,[ 0 250],[ 0 250],268.795665,231.432983
2,4063,582,15.0,3,2016-07-20 03:09:30,2016-07-20 03:43:30,0 days 00:34:00.000000000,11.783160,125.498200,12436367504177285501,...,10.545011,110.205700,16962820405680796987,2016-07-20 03:43:45.639048,0,0,[350 250],[350 250],360.404397,367.113247
3,4274,1506,26.0,5,2016-08-15 13:32:00,2016-08-15 13:54:00,0 days 00:22:00.000000000,217.782600,96.482190,10765467797736549964,...,141.507900,225.099500,17506577338879220375,2016-08-15 13:53:59.925208,2,1,[350 250],[ 0 250],202.605920,143.682012
4,1174,220,27.0,5,2016-07-26 14:55:30,2016-07-26 15:07:00,0 days 00:11:30.000000000,106.111200,176.806200,12629584191723889970,...,4.070043,216.114800,15879263399086570856,2016-07-26 15:07:18.033893,2,1,[ 0 250],[ 0 250],128.906629,34.128757
5,220,77,28.0,5,2016-07-26 03:29:00,2016-07-26 03:43:00,0 days 00:14:00.000000000,70.498990,144.239300,12696127842670122872,...,75.740500,128.994000,17809985327667396760,2016-07-26 03:43:21.149826,2,1,[ 0 250],[ 0 250],127.104025,142.755299
6,18135,2970,5.0,1,2016-08-16 21:50:00,2016-08-16 21:59:00,0 days 00:09:00.000000000,179.178200,44.282540,18179642034074970171,...,170.505600,35.535980,15755472128928802248,2016-08-16 21:59:33.502621,3,1,[ 0 250],[ 0 250],272.808542,273.983531
7,21057,1702,3.0,0,2016-07-24 19:40:00,2016-07-24 20:07:00,0 days 00:27:00.000000000,259.326600,96.004450,10002761260679133725,...,276.288900,96.414110,9644668760040799924,2016-07-24 20:07:10.286227,1,0,[350 250],[350 250],178.707288,170.358304
8,5573,1667,25.0,5,2016-08-15 03:59:30,2016-08-15 04:52:30,0 days 00:53:00.000000000,235.389000,127.500000,10543061345195463491,...,238.814000,36.791480,14664115474002021663,2016-08-15 04:52:49.995149,1,0,[350 250],[350 250],167.755570,240.458312
9,4290,1308,25.0,5,2016-08-08 07:54:00,2016-08-08 08:38:30,0 days 00:44:30.000000000,330.171800,170.953100,12711765534957691618,...,245.480500,147.991200,12859521269969720602,2016-08-08 08:38:52.908942,1,0,[350 250],[350 250],81.495828,146.048352


In [ ]:
# GENERATE MUTLIPLE VIDEOS
for i, row in gaps.iterrows():
    try:
        generate_presence_video(row)
    except (ValueError, TypeError):
        print('error, skip one')
        continue

2175 - 2016-07-31 02:52:00 already exists, returning with no result.
1395 - 2016-07-23 04:48:30 already exists, returning with no result.
582 - 2016-07-20 03:09:30 already exists, returning with no result.
1506 - 2016-08-15 13:32:00 already exists, returning with no result.
220 - 2016-07-26 14:55:30 already exists, returning with no result.
77 - 2016-07-26 03:29:00 already exists, returning with no result.
2970 - 2016-08-16 21:50:00 already exists, returning with no result.
1702 - 2016-07-24 19:40:00 already exists, returning with no result.
1667 - 2016-08-15 03:59:30 already exists, returning with no result.
1308 - 2016-08-08 07:54:00 already exists, returning with no result.
3148 - 2016-09-12 19:29:30 already exists, returning with no result.
392 - 2016-08-10 01:59:30 already exists, returning with no result.
2181 - 2016-07-31 22:00:30 already exists, returning with no result.
2062 - 2016-08-04 05:53:30 already exists, returning with no result.
2838 - 2016-08-10 19:28:30 already exis

error, skip one
2016-07-24 05:33:00 666
after frames
unm_unf_s666_02_1548_random_exit


2016-07-27 02:31:30 303
after frames
unm_unf_s666_02_1826_random_exit


2016-07-25 08:32:30 1025
after frames
unm_unf_s666_02_1827_random_exit


2016-07-24 09:46:30 1173
after frames
unm_unf_s666_02_1653_random_exit


2016-08-14 12:54:30 1549
after frames
unm_unf_s666_02_2198_random_exit


2016-08-02 07:27:30 895
after frames
unm_unf_s666_02_2243_random_exit


2016-08-13 18:57:00 2274
after frames
unm_unf_s666_02_1362_random_exit


2016-08-01 03:02:00 364
after frames
unm_unf_s666_02_1911_random_exit


In [ ]:
gaps.shape

In [ ]:
# Would be nice to see the presence(s) in context, I guess?

In [ ]:
# plot_presence(presx)
# plot_presence(presbin)

In [31]:
#%% FUNCTIONS
def generate_presence_video(row):
    bee, timestamp, frame = row.bee_id, row.exit, row.exit_frame 
    cam_id = row.cam_id_exit
    dist = row.hiveexit_distance_for_exit 
    # ASK: can this be used given hiccups? or just take days w/o hiccups?
    interval_margin = 2
    frames_per_interval = interval_size*fps
    frame_margin = interval_margin*frames_per_interval #30s * 3FPS
    name = 'unm_unf_s666_' + conf_string + '_' + str(bee) + '_random_exit'
    path = '/home/mi/rrszynka/mnt/janek/caches/Videos/' + name + ".mp4"
    if os.path.isfile(path):
        print(str(bee) + ' - ' + str(timestamp) + ' already exists, returning with no result.')
        return
    video = plot.plot_bees(bee_ids=[bee],
                       frame_id=frame, frame_margin=frame_margin, # TEMP: should be frame_margin
                       path_alpha=None, bt_export=None, # '/home/mi/rrszynka/mnt/janek/caches/Videos/bt_export/'+name+".json",
                       plot_labels = True, plot_markers=True)

    intervals = len(video._frames)/(interval_margin*2)

    #TODO: check this conversion from timestamp to interval against the conversion that goes the other way
    middle_timestamp = timestamp

    secs = time_to_secs(middle_timestamp)
    interval_index_mid = int(secs/30)
    print(timestamp, interval_index_mid)
    start_interval = interval_index_mid - interval_margin

    _, presence_df = next(((d, p) for d , p in presences if d == timestamp.date()), None)
    _, bin_presence_df = next(((d, p) for d , p in presences_bin if d == timestamp.date()), None)


    pres_row = presence_df.loc[bee]
    bin_pres_row = bin_presence_df.loc[bee]

    for i, frame in enumerate(video._frames):
        interval_index = (i // frames_per_interval)
        #PRINT: pres_score, pres_bin, cam_id, bee_id (or save in file), exit_dist(would be nice)
        frame._title = "\n c: " + str(cam_id) +", d:" + str(int(dist)) + ",  s: " + str(pres_row[start_interval + interval_index]) + ",  p: " + str(bin_pres_row[start_interval + interval_index])
    print('after frames')
    print(name)
    video._crop_margin = None
    video.get_video(save_to_path='/home/mi/rrszynka/mnt/janek/caches/Videos/' + name + ".mp4")

In [7]:
def time_to_secs(time):
    return time.hour * 3600 + time.minute * 60 + time.second

    
def binarize_presence(presence_for_day_df):
    # TODO: running this on an already-binarized series will return all zeros,
    # (and that's not what we want) - changing the threshold solves it, but try sth else

    ys = presence_for_day_df.copy()
    index = ys.index
    if ys[ys>1].sum().sum() == 0:
        #Consider this already binarized, make no changes
        return ys

    ys[ys>90] = 90
    ys[ys>45] = 90
    ys[ys<=45] = 0 #TODO: consult: what should the threshold be
    ys[ys==90] = 1
    ys = closing(ys, rectangle(1,15))
    ys = pd.DataFrame(ys, index=index)
    ys.index.name = 'bee_id'
    return ys